In [25]:
import os

import cv2
import numpy as np
import pandas as pd
from keras.preprocessing import image
import keras.applications
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Conv2D, Flatten, BatchNormalization
from keras.utils import print_summary
from keras import regularizers
from keras.callbacks import TensorBoard
from keras import Model
import keras.callbacks
import tensorflow
from sklearn.metrics import classification_report

In [2]:
VAL_LABELS_FILE = "/home/jurek/Pulpit/zajecia/SUKIENKI/val/labels.txt"
TRAIN_LABELS_FILE = "/home/jurek/Pulpit/zajecia/SUKIENKI/train/labels.txt"
TEST_LABELS_FILE="/home/jurek/Pulpit/zajecia/SUKIENKI/test/labels.txt"

In [3]:
df = pd.read_csv('/home/jurek/Pulpit/zajecia/SUKIENKI/train/labels.txt', delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")


In [34]:

def get_generator(filename, number=None, shuffle=True):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv(filename, delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", class_mode="other", y_col=['beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], target_size=(128, 128), batch_size=40, shuffle=shuffle)

def show_dataset(flow):
    for img, l in flow:
        # wypisz etykietę
        print(l[0])
        # konwersja obrazu do wyświetlenia
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # wyświetl obraz
        cv2.imshow("image", img)
        # poczekaj na klawisz
        cv2.waitKey()


In [35]:
def create_model(gen):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(128,128, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(52, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(14, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model

In [36]:
def get_callbacks():
    callbacks =[]
    mc=keras.callbacks.ModelCheckpoint("weights.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    tb = TensorBoard()
    callbacks.append(mc)
    callbacks.append(tb)
  
    return callbacks

In [37]:
def test_model(model):
    test_gen = get_generator(TEST_LABELS_FILE)
    results = model.evaluate_generator(test_gen)
    for name, result in zip(model.metrics_names, results):
        print(f"{name}: {result}")

In [38]:
def get_gt_from_generator(gen):
    gt = []
    for _ in range(len(gen)):
        _, labels= next(gen)
        gt.extend(labels)
    gen.reset()
    return np.array(gt).astype(np.uint8)

In [39]:
def show_report(model):
    test_gen = get_generator(TEST_LABELS_FILE)
    gt = get_gt_from_generator(test_gen)
    pred = model.predict_generator(test_gen)
    pred = np.round(pred).astype(np.uint8)

    report = classification_report(gt ,pred)
    print(report)


In [40]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE)
    val_gen = get_generator(VAL_LABELS_FILE)
    callbacks = get_callbacks()
    # stwórz model
    model = create_model(train_gen)
    model.load_weights('weights.hdf5')
    model.summary()
    # trening :)
    #model.fit_generator(train_gen, steps_per_epoch=train_gen.n // train_gen.batch_size, epochs=30, callbacks=callbacks,
     #                  validation_data=val_gen, validation_steps=val_gen.n // val_gen.batch_size)

    test_model(model)    
    show_report(model)   

In [41]:
main()

Found 3514 validated image filenames.
Found 449 validated image filenames.
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization_31 (Batc (None, 126, 126, 16)      64        
_________________________________________________________________
activation_31 (Activation)   (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 124, 124, 16)      2320      
_________________________________________________________________
batch_normalization_32 (Batc (None, 124, 124, 16)      64        
_________________________________________________________________
activation_32 (Activation)   (None, 124, 124, 16)      0         
_____________________________________________

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [3]:
from keras.applications.inception_v3 import InceptionV3

from keras.layers import Input

In [9]:

def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv(filename , delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", class_mode="other", y_col=['beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], target_size=(128, 128), batch_size=40)

def show_dataset(flow):
    for img, l in flow:
        # wypisz etykietę
        print(l[0])
        # konwersja obrazu do wyświetlenia
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # wyświetl obraz
        cv2.imshow("image", img)
        # poczekaj na klawisz
        cv2.waitKey()


In [10]:
def create_model(gen):


# this could also be the output a different Keras model or layer
      # this assumes K.image_data_format() == 'channels_last'

    model = InceptionV3(weights='imagenet', input_shape=(128,128,3), include_top=False, pooling='avg')
    dense = Dense(14, activation='sigmoid')(model.output) 
    model = Model(inputs=[model.input], outputs=[dense])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model

In [11]:
def get_callbacks():
    callbacks =[]
    mc=keras.callbacks.ModelCheckpoint("weights1.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    callbacks.append(mc)
    return callbacks

In [16]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE)
    val_gen = get_generator(VAL_LABELS_FILE)
    callbacks = get_callbacks()
    # stwórz model
    model = create_model(train_gen)
    #model.load_weights('weights1.hdf5')
    model.summary()
    # trening :)
    model.fit_generator(train_gen, epochs=30, steps_per_epoch=train_gen.n // train_gen.batch_size,
                        validation_data = val_gen, validation_steps = val_gen.n // val_gen.batch_size,
                        callbacks=callbacks)

In [17]:
main()

Found 3514 validated image filenames.
Found 0 validated image filenames.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_260 (Conv2D)             (None, 63, 63, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_260 (BatchN (None, 63, 63, 32)   96          conv2d_260[0][0]                 
__________________________________________________________________________________________________
activation_259 (Activation)     (None, 63, 63, 32)   0           batch_normalization_260[0][0]    
___________________

Epoch 1/30
47/87 [===============>..............] - ETA: 1:58 - loss: 0.2390 - binary_accuracy: 0.9114

KeyboardInterrupt: 